# Code explanation for the avg_yy_strain-vs-separation folder

## Requirements

- Needs to have a **dump.file** inside the **avg_yy_strain-vs-separation** folder to run the code. *You can also change the path in the **main.py** file.*
- Needs **function.py** and **rewrite_dump.py** in the same folder as **main.py** to run.
- Needs `matplotlib` to print graphs. Run the following command in the terminal to install the library: `pip install matplotlib`.
- Needs `linecache` to print read the file. Run the following command in the terminal to install the library: `pip install linecache`.

## function.py

This file contains all the functions used by the **main.py** file. Each functions are commented. 

In [ ]:
import linecache
import numpy as np

def split_atom_info(info):
    """Atom info string -> Atom info list"""
    res = info.strip('\n')
    atom = res.split(' ')
    return atom

def count_nb_line(file):
    """Count the number of line of 1 dump file"""
    fd = open(file,'r')
    n=0
    for line in fd:
        n += 1
    fd.close()
    return n


def extract_atom_type(file, type):
    """Extract atom type from a dump.file for ONE TIMESTEP ! Return an array"""
    atom_type_1 = []
    nb_atoms = int(linecache.getline(file,4))
    nb_line = nb_atoms+9

    for i in range(1,nb_line+1):
        atom_info = split_atom_info(linecache.getline(file,i))
        if len(atom_info) == 1:
            continue
        elif atom_info[1] == str(type):
            atom_type_1.append(atom_info + [i]) #i correspond to the line number which would be helpful for later
    return np.array(atom_type_1)

def define_box(a_0, Delta_X, Delta_Y):
    """Return Box coordinates"""
    Y_MIN = 10*a_0
    Y_MAX = Y_MIN + Delta_Y*a_0
    X_MIN = 10*a_0
    X_MAX = X_MIN + Delta_X*a_0
    return Y_MIN, Y_MAX, X_MIN, X_MAX

def atoms_in_box(atoms_array, Y_MIN, Y_MAX, X_MIN, X_MAX):
    """Return an array with all atoms in the box"""
    atoms_in = []
    for i in range(0,len(atoms_array)):
        if float(atoms_array[i][2]) >= X_MIN and float(atoms_array[i][2]) <= X_MAX:
            if float(atoms_array[i][3]) >= Y_MIN and float(atoms_array[i][3]) <= Y_MAX:
                atoms_in.append(atoms_array[i])
    return np.array(atoms_in)

def new_atoms_info(file, array_atom_in, nb_atoms, ite):
    """Return the atoms that were in the box new information as an array"""
    lines_to_go = []
    for i in range(len(array_atom_in)):
        lines_to_go.append(int(array_atom_in[i][8])+ite*(nb_atoms+9))

    res = []
    for line in lines_to_go:
        res.append(split_atom_info(linecache.getline(file, line)))
    return np.array(res)

def calc_avg_yy_strain(array_atoms_in, nb_atoms, Delta_X, a_0):
    """Return the average yy strain"""
    sum = 0
    for i in range(0,len(array_atoms_in)):
        sum += float(array_atoms_in[i][6])
    fy_avg = sum*(1/nb_atoms)
    sy_avg = fy_avg/(Delta_X*a_0)
    return sy_avg

def calc_separation(array_atoms_in, Delta_Y, a_0):
    """Return the separation"""
    L_0 = Delta_Y*a_0
    Y_list = []
    for i in range(0,len(array_atoms_in)):
        Y_list.append(float(array_atoms_in[i][3]))
    sep = max(Y_list) - min(Y_list) - L_0
    return sep

## rewrite_dump.py

This file contains the function `trier_fichier` used to rewrite in a correct order **dump.file**.

In [ ]:
# coding: utf-8

import numpy as np
from pandas import DataFrame

def trier_fichier(infile, output_file, timesteps):
    """Sorts the dump.file by atom id in order to have the same structure. Creates an output file."""

    # nombre d'atomes
    input_file = open(infile, 'r')
    for i in range(3):
        input_file.readline()
    nb_atoms = int(input_file.readline().strip('\n'))
    input_file.close()


    # tri de tous les timesteps du fichier
    with open(infile, 'r') as input_file:
        with open(output_file, 'w') as output_file:
            for k in range(timesteps):
                data = []
                #réécrit l'en-tête
                for i in range(9):
                    line = input_file.readline()
                    output_file.write(line)
                for i in range(nb_atoms):
                    data.append(input_file.readline().strip('\n').split())
                data = DataFrame(data)

            #  print(type(data.iloc[0,0]))
                data[0] = data[0].astype(int)
                data = data.sort_values(by=[0])
                data[0] = data[0].astype(str)
                data = np.array(data)
                
            #  print(data)
                for i in data:
                    output_file.write(' '.join(i)+'\n')
    return output_file

## main.py

This file contains the code that runs functions in the correct order to print a final graph. This code is self-explanatory thanks to the comments. 

In [ ]:
from function import *
import matplotlib.pyplot as plt
from rewrite_dump import trier_fichier

# Define variables
filename = r'avg_yy_strain-vs-separation/dump.test'
nb_atoms = int(linecache.getline(filename,4))
nb_timestep = int(count_nb_line(filename)/(nb_atoms+9))
trier_fichier(filename, r'avg_yy_strain-vs-separation/sorted.file', nb_timestep) #File sorting to always have the same structure
sorted_dump = r'avg_yy_strain-vs-separation/sorted.file'
a_0 = 0.93
Delta_X = 4
Delta_Y = 8
Y_MIN, Y_MAX, X_MIN, X_MAX = define_box(a_0, Delta_X, Delta_Y)
list_avg_yy_strain = []
list_separation = []

# Finding atoms in the box
arr_atom_type = extract_atom_type(sorted_dump, 1)
atoms_inside = atoms_in_box(arr_atom_type, Y_MIN, Y_MAX, X_MIN, X_MAX)
print(atoms_inside)


# Looping on all the TIMESTEPS
for j in range(nb_timestep):
    atoms_info = new_atoms_info(sorted_dump, atoms_inside, nb_atoms, j) #Actualing atoms position and force that were presents in the box
    sy_avg = calc_avg_yy_strain(atoms_info, nb_atoms, Delta_X, a_0)
    sep = calc_separation(atoms_info, Delta_Y, a_0)
    list_avg_yy_strain.append(sy_avg)
    list_separation.append(sep)

# Plot
plt.scatter(list_separation, list_avg_yy_strain) #Scatter -> Nuage de points
plt.xlabel('separation')
plt.ylabel('average yy strain')
plt.title('total cohesive law based on ' + str(nb_timestep) + ' timesteps')
plt.show()